In [3]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('ggplot')

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import LSTM


from keras.models import Sequential, Model, Input
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import LSTM
import sklearn
from sklearn import preprocessing, model_selection
import tensorflow as tf

import src.lstm_helper as helper

In [45]:
#reference:https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
# https://heartbeat.fritz.ai/building-a-neural-network-from-scratch-using-python-part-1-6d399df8d432

### Predict future sales but non time-series models.

#### Fearture engineering

In [5]:
path = 'data/processed_train.csv'
df = pd.read_csv(path)
df.shape

(913000, 13)

In [6]:
df.head()

,date,store,item,sales,year,month,day,day_of_week,year_and_month,weekend,quarter,week_block_num,quarter_block_num
0,2013-01-01,1,1,13,2013,1,1,1,2013-01,0,1,1,1
1,2013-01-02,1,1,11,2013,1,2,2,2013-01,0,1,1,1
2,2013-01-03,1,1,14,2013,1,3,3,2013-01,0,1,1,1
3,2013-01-04,1,1,13,2013,1,4,4,2013-01,0,1,1,1
4,2013-01-05,1,1,10,2013,1,5,5,2013-01,1,1,1,1


In [8]:
#set datatime to index
# df['date'] = pd.to_datetime(df['date'])
# df = df.set_index('date')
# df.head()

In [7]:
df.iloc[500,:]

date                 2014-05-16
store                         1
item                          1
sales                        23
year                       2014
month                         5
day                          16
day_of_week                   4
year_and_month          2014-05
weekend                       0
quarter                       2
week_block_num               72
quarter_block_num             6
Name: 500, dtype: object

In [8]:
#proceese data
processed_df = helper.data_pipeline(path)

In [9]:
processed_df.shape

(913000, 69)

In [10]:
processed_df.head()

,date,store,item,sales,month,day,day_of_week,weekend,quarter,week_block_num,...,i_41,i_42,i_43,i_44,i_45,i_46,i_47,i_48,i_49,i_50
0,2013-01-01,1,1,13,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2013-01-02,1,1,11,1,2,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2013-01-03,1,1,14,1,3,3,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2013-01-04,1,1,13,1,4,4,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2013-01-05,1,1,10,1,5,5,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#drop unique store & item Id
drop_cols =['store','item']

In [12]:
processed_df=processed_df.drop(columns=drop_cols,axis =1)

In [13]:
#set datatime to index
processed_df['date'] = pd.to_datetime(processed_df['date'])
processed_df = processed_df.set_index('date')
processed_df.head()

,sales,month,day,day_of_week,weekend,quarter,week_block_num,quarter_block_num,s_2,s_3,...,i_41,i_42,i_43,i_44,i_45,i_46,i_47,i_48,i_49,i_50
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,13,1,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-01-02,11,1,2,2,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-01-03,14,1,3,3,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-01-04,13,1,4,4,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-01-05,10,1,5,5,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
#split into train and valid sets
train = processed_df.loc['2013-01-01':'2017-09-30']
valid = processed_df.loc['2017-10-01':'2017-12-31']

In [33]:
y_train = train['sales'].values.reshape(-1,1)
X_train = train.drop('sales',axis=1)
y_test  = valid['sales'].values.reshape(-1,1)
X_test = valid.drop('sales',axis=1)

In [34]:
print(y_train.shape)
print(X_train.shape)
print(y_test.shape)
print(X_test.shape)

(867000, 1)
(867000, 65)
(46000, 1)
(46000, 65)


In [35]:
processed_df.sales.sum() == (sum(y_test)+sum(y_train))

array([ True])

### Normalize

In [36]:
#standardize the dataset
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)


In [38]:
print(f"Shape of train set is {X_train.shape}")
print(f"Shape of test set is {X_test.shape}")
print(f"Shape of train label is {y_train.shape}")
print(f"Shape of test labels is {y_test.shape}")

Shape of train set is (867000, 65)
Shape of test set is (46000, 65)
Shape of train label is (867000, 1)
Shape of test labels is (46000, 1)


In [40]:
type(y_train),type(X_train)

(numpy.ndarray, numpy.ndarray)

In [41]:
y_train[:5]

array([[13],
       [11],
       [14],
       [13],
       [10]])

array([[-1.56779261, -1.67364947, -1.00072166, -0.63207257, -1.31072375,
        -1.72704986, -1.64550306, -0.33333333, -0.33333333, -0.33333333,
        -0.33333333, -0.33333333, -0.33333333, -0.33333333, -0.33333333,
        -0.33333333, -0.14285714, -0.14285714, -0.14285714, -0.14285714,
        -0.14285714, -0.14285714, -0.14285714, -0.14285714, -0.14285714,
        -0.14285714, -0.14285714, -0.14285714, -0.14285714, -0.14285714,
        -0.14285714, -0.14285714, -0.14285714, -0.14285714, -0.14285714,
        -0.14285714, -0.14285714, -0.14285714, -0.14285714, -0.14285714,
        -0.14285714, -0.14285714, -0.14285714, -0.14285714, -0.14285714,
        -0.14285714, -0.14285714, -0.14285714, -0.14285714, -0.14285714,
        -0.14285714, -0.14285714, -0.14285714, -0.14285714, -0.14285714,
        -0.14285714, -0.14285714, -0.14285714, -0.14285714, -0.14285714,
        -0.14285714, -0.14285714, -0.14285714, -0.14285714, -0.14285714]])

In [27]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(processed_df)

In [35]:
training_set_scaled[:1]

array([[0.05627706, 0.        , 0.        , 0.16666667, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In [29]:
training_set_scaled.shape

(913000, 66)

In [48]:
#split into train and valid sets
train = training_set_scaled[:-46000]
valid = training_set_scaled[-46000:]

In [50]:
train.shape

(867000, 66)

In [55]:
y_train = train[:,0]
X_train = train[:,1:]
y_test  = valid[:,0]
X_test = valid[:,1:]

print(y_train.shape)
print(X_train.shape)
print(y_test.shape)
print(X_test.shape)

(867000,)
(867000, 65)
(46000,)
(46000, 65)


In [56]:
y_train[:5]

array([0.05627706, 0.04761905, 0.06060606, 0.05627706, 0.04329004])

#### Neural Networks

In [ ]:
#### Neural Networks

In [57]:
model = Sequential()

In [63]:
model.add(Dense(10, input_shape=(65,)))

TypeError: The added layer must be an instance of class Layer. Found: <tensorflow.python.keras.layers.core.Dense object at 0x7f8a0df3dc50>

In [59]:
model.compile(loss='mean_squared_error',
             optimizer='sgd')

In [60]:
model.fit(X_train, y_train, epochs=5, batch_size=32)

ValueError: Error when checking target: expected sequential_2_input to have shape (65,) but got array with shape (1,)